In [1]:
%run scripts/helper.py
%run scripts/models.py

In [2]:
crowd_train = load_file('./data/train.csv/train.csv', index_col='id')

In [3]:
y = crowd_train.median_relevance.values

In [5]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import make_scorer

In [6]:
kappa_scorer = make_scorer(quadratic_weighted_kappa, greater_is_better = True)

In [7]:
X = tweak_text(crowd_train)

C:\Anaconda\lib\site-packages\bs4\__init__.py:176: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65497012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
C:\Anaconda\lib\site-packages\bs4\__init__.py:176: UserWarning: "http://i104.photobucket.com/albums/m175/champions_on_display/wincraft2013/januaryb/65516012.jpg" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL,

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

In [9]:
tfv = TfidfVectorizer(min_df=3, max_df=500, max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

svd = TruncatedSVD(n_components=200, algorithm='randomized', n_iter=5, random_state=None, tol=0.0)
scl = StandardScaler(copy=True, with_mean=True, with_std=True)

In [10]:
X_tfv = tfv.fit_transform(X)
X_svd = svd.fit_transform(X_tfv)

In [22]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute')

In [21]:
scores = cross_val_score(knn, X_svd, y, scoring=kappa_scorer, n_jobs=1, cv=2)
print 'min score %0.4f, max score %0.4f and mean score %0.4f ' %(scores.min(), scores.max(), scores.mean())

min score 0.5001, max score 0.5005 and mean score 0.5003 


In [17]:
tweaked_model, tfv, svd = build_knn_model(X, y, 'distance')

In [29]:
crowd_test = load_file('./data/test.csv/test.csv', index_col='id')

In [30]:
Xtest = tweak_text(crowd_test)

In [32]:
predictions = []
for i in range(0, len(Xtest), 5000):
    preds = knn_model_predictions(tweaked_model, tfv, svd, Xtest[i:i+5000])
    predictions.append(preds)

In [33]:
all_preds = np.hstack(predictions)

In [34]:
all_preds.shape

(22513,)

In [35]:
make_submission(crowd_test.index.values.astype(int), all_preds, './model-submissions/knn_distance_processed.csv')